In [1]:
# import wandb
import argparse
# wandb.login()

In [2]:
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="hello-world-deep-learning",

#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.001,
#     "architecture": "CNN",
#     "dataset": "CIFAR-10",
#     "epochs": 1,
#     }
# )


import packages and define constant

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# transform before training
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [4]:
trainset, validset = torch.utils.data.random_split(
    torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform = transform),
    lengths=[40000, 10000])

# trainset = torchvision.datasets.CIFAR100(root='./data', train=True,transform = torchvision.transforms.Compose([
#     torchvision.transforms.RandomCrop(size=(32, 32), padding = 4),
#     torchvision.transforms.RandomHorizontalFlip(),
#     torchvision.transforms.ToTensor(),
#     torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
# ]), download = True)

# validset = torchvision.datasets.CIFAR100(root='./data', train=True,transform = torchvision.transforms.Compose([
#     torchvision.transforms.RandomCrop(size=(32, 32), padding = 4),
#     torchvision.transforms.RandomHorizontalFlip(),
#     torchvision.transforms.ToTensor(),
#     torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
# ]), download = True)

# testset = torchvision.datasets.CIFAR100(root='./data', train=False,transform = torchvision.transforms.Compose([
#     torchvision.transforms.RandomCrop(size=(32, 32), padding = 4),
#     torchvision.transforms.RandomHorizontalFlip(),
#     torchvision.transforms.ToTensor(),
#     torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
# ]), download = True)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=128, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)
classes = ('apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle',
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel',
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock',
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster',
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm')


Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Your deep neural network
class ResBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResBlock, self).__init__()
        self.block_conv = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size = 3, stride = stride, padding = 1),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(),
            nn.Conv2d(outchannel, outchannel, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out1 = self.block_conv(x)
        out2 = self.shortcut(x) + out1
        out2 = F.relu(out2)
        return out2


class ResNet_18(nn.Module):
    def __init__(self, ResBlock, num_classes):
        super(ResNet_18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 7, stride = 2, padding = 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        )

        self.layer1=self.make_layer(ResBlock, 64, 2, stride = 1)
        self.layer2 = self.make_layer(ResBlock, 128, 2, stride = 2)
        self.layer3 = self.make_layer(ResBlock, 256, 2, stride = 2)
        self.layer4 = self.make_layer(ResBlock, 512, 2, stride = 2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        #下面一行是bili
        self.fc = nn.Linear(512, num_classes)
        #self.linear = nn.Linear(512 * 1 * 1,100)
        #self.dropout = nn.Dropout(0.3)
        
    def make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        
        #下面兩行是bili
        x = torch.flatten(x, 1)
        x = self.fc(x)
        #x = x.view(x.size(0), -1)
        #x = self.linear(x)
        #x = self.dropout(x)
        return x

net = ResNet_18(ResBlock, num_classes = 100).to(device)

# optimizer and loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.01, momentum = 0.9)


In [6]:
# start training
from tqdm.notebook import tqdm
training_time = 250
progress_bar = tqdm(enumerate(trainloader, 0), total=len(trainloader) * training_time, desc='Training' )

for epoch in range(training_time):  # epoch
    print("開始訓練第", epoch + 1, "輪")
    running_loss = 0.0
    for i, data in (enumerate(trainloader, 0)):

        inputs, labels = data[0].to(device), data[1].to(device)

        # zero gradient
        optimizer.zero_grad()

        # forward, backward, optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #wandb.log({"loss": loss})
        progress_bar.update(1)
        running_loss += loss.item()
        if i % 250 == 0:
            # validation
            net.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for data in validloader:
                    images, labels = data[0].to(device), data[1].to(device)
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            progress_bar.set_postfix(accuracy=100 * correct / total)
            #wandb.log({"accuracy": correct/total})
        running_loss = 0.0

print('Finished Training')
torch.save(net, './models/ResNet18.pt')

Training:   0%|          | 0/78250 [00:00<?, ?it/s]

開始訓練第 1 輪
開始訓練第 2 輪
開始訓練第 3 輪
開始訓練第 4 輪
開始訓練第 5 輪
開始訓練第 6 輪
開始訓練第 7 輪
開始訓練第 8 輪
開始訓練第 9 輪
開始訓練第 10 輪
開始訓練第 11 輪
開始訓練第 12 輪
開始訓練第 13 輪
開始訓練第 14 輪
開始訓練第 15 輪
開始訓練第 16 輪
開始訓練第 17 輪
開始訓練第 18 輪
開始訓練第 19 輪
開始訓練第 20 輪
開始訓練第 21 輪
開始訓練第 22 輪
開始訓練第 23 輪
開始訓練第 24 輪
開始訓練第 25 輪
開始訓練第 26 輪
開始訓練第 27 輪
開始訓練第 28 輪
開始訓練第 29 輪
開始訓練第 30 輪
開始訓練第 31 輪
開始訓練第 32 輪
開始訓練第 33 輪
開始訓練第 34 輪
開始訓練第 35 輪
開始訓練第 36 輪
開始訓練第 37 輪
開始訓練第 38 輪
開始訓練第 39 輪
開始訓練第 40 輪
開始訓練第 41 輪
開始訓練第 42 輪
開始訓練第 43 輪
開始訓練第 44 輪
開始訓練第 45 輪
開始訓練第 46 輪
開始訓練第 47 輪
開始訓練第 48 輪
開始訓練第 49 輪
開始訓練第 50 輪
開始訓練第 51 輪
開始訓練第 52 輪
開始訓練第 53 輪
開始訓練第 54 輪
開始訓練第 55 輪
開始訓練第 56 輪
開始訓練第 57 輪
開始訓練第 58 輪
開始訓練第 59 輪
開始訓練第 60 輪
開始訓練第 61 輪
開始訓練第 62 輪
開始訓練第 63 輪
開始訓練第 64 輪
開始訓練第 65 輪
開始訓練第 66 輪
開始訓練第 67 輪
開始訓練第 68 輪
開始訓練第 69 輪
開始訓練第 70 輪
開始訓練第 71 輪
開始訓練第 72 輪
開始訓練第 73 輪
開始訓練第 74 輪
開始訓練第 75 輪
開始訓練第 76 輪
開始訓練第 77 輪
開始訓練第 78 輪
開始訓練第 79 輪
開始訓練第 80 輪
開始訓練第 81 輪
開始訓練第 82 輪
開始訓練第 83 輪
開始訓練第 84 輪
開始訓練第 85 輪
開始訓練第 86 輪
開始訓練第 87 輪
開始訓練第 88 輪
開始訓練第 89 輪
開始訓練第 90 輪
開始訓練第 91 輪
開始訓練第 92

In [7]:
# test network on test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

# calculate accuracy
class_correct = list(0. for i in range(100))
class_total = list(0. for i in range(100))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(100):
    correctRate = 0.0
    if (class_total[i] != 0):
        correctRate = class_correct[i] / class_total[i]
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * correctRate))


Accuracy of the network on the 10000 test images: 41 %
Accuracy of apple : 100 %
Accuracy of aquarium_fish :  0 %
Accuracy of  baby : 66 %
Accuracy of  bear :  0 %
Accuracy of beaver :  0 %
Accuracy of   bed : 60 %
Accuracy of   bee :  0 %
Accuracy of beetle :  0 %
Accuracy of bicycle :  0 %
Accuracy of bottle : 66 %
Accuracy of  bowl : 33 %
Accuracy of   boy : 33 %
Accuracy of bridge : 16 %
Accuracy of   bus : 100 %
Accuracy of butterfly : 28 %
Accuracy of camel : 25 %
Accuracy of   can : 50 %
Accuracy of castle :  0 %
Accuracy of caterpillar :  0 %
Accuracy of cattle :  0 %
Accuracy of chair : 80 %
Accuracy of chimpanzee : 33 %
Accuracy of clock :  0 %
Accuracy of cloud : 50 %
Accuracy of cockroach :  0 %
Accuracy of couch :  0 %
Accuracy of  crab : 50 %
Accuracy of crocodile :  0 %
Accuracy of   cup : 50 %
Accuracy of dinosaur : 40 %
Accuracy of dolphin : 25 %
Accuracy of elephant : 25 %
Accuracy of flatfish : 25 %
Accuracy of forest : 28 %
Accuracy of   fox : 100 %
Accuracy of  gir

In [8]:
#wandb.finish()